In [1]:
import numpy as np
import matplotlib.pyplot as plt
from database_methods import Table, to_show, head, process_data
import sqlite3
con = sqlite3.connect('database_results.db')
cursor = con.cursor()

In [2]:
mas = Table(connection=con, name='masters')
exp = Table(con, name = 'experiments')
meas = Table(con, name = 'measurements')

In [3]:
print(mas.columns)
print(exp.columns)
print(meas.columns)
print('-'*80)
mas.show_all()
exp.show_all()

CATEGORY, PERIOD, AMPLITUDE, MASTER, FWHM, R, S
DATE, MASTER_ID, DIP, FOLDER
EXPERIMENT_ID, NP, WAVELENGTH, FILE, AR_TH, AR_DIP, AR_NOISE, EXC_THS, RATE, INFO, COUNT, X, Y, CON, INP, INPPX, IBGPX, STD, stdX, stdY, stdCON, stdINP, stdINPPX, stdIBGPX, stdSTD, NOTES, AR_MIN, AR_COND, EXP_TYPE, OPT
--------------------------------------------------------------------------------
ID	CATEGORY	PERIOD	AMPLITUDE	MASTER	FWHM	R	S
1	600	405	45	M5	23.0	8.0	15.9
2	600	405	53	P3	25.0	19.7	3.8
3	650	452	32	K4	12.5	11.9	17.8
4	650	452	40	K5	15.0	18.0	8.8
5	750	528	24	L3	7.0	13.8	78.5
6	750	526	30	Q4	9.5	22.0	30.7
ID	DATE	MASTER_ID	DIP	FOLDER
1	20-02-25	2	615	20_02_25_P3/
2	20-02-26	6	727	20_02_26_Q3/
3	20-03-16	4	658	20_03_16_K5/
5	20-02-25	1	598	20_02_25_M5/
6	20-03-16	3	647	20_03_16_K4/
7	20-04-15	5	740	20_04_15_L3/
8	20-04-14	1	606	20_04_14_M5/
9	20-04-20	6	729	20_04_20_Q4/


In [4]:
con.commit()

In [22]:
# 
columns = 'MASTER, DATE, S, AVG(DIP), AVG(X), AVG(Y), AVG(CON), AVG(stdX), AVG(stdY), AVG(stdCON), AVG(INP), AVG(INPPX), AVG(INPPX), AVG(IBGPX)'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE NP = 80 AND OPT = 1
GROUP BY MASTER
ORDER BY DIP DESC
;
   """.format(columns))

head(columns)

for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])


MASTER	DATE	S	AVG(DIP	AVG(X)	AVG(Y)	AVG(CON	AVG(std	AVG(std	AVG(std	AVG(INP	AVG(INP	AVG(INP	AVG(IBG
L3	20-04-1	78.50	740.00	57.11	9.82	3.11	22.00	1.35	0.8999	19.64	0.9066	0.9066	0.2933
Q4	20-04-2	30.70	729.00	25.58	9.68	10.18	8.12	1.51	4.65	26.32	2.85	2.85	0.2820
K5	20-03-1	8.80	658.00	20.66	12.63	5.17	3.84	1.57	2.31	15.64	1.44	1.44	0.2827
K4	20-03-1	17.80	647.00	18.97	12.46	4.27	2.46	1.40	1.36	9.48	1.15	1.15	0.2686
P3	20-02-2	3.80	615.00	5.38	6.25	7.52	1.42	1.64	2.18	4.90	1.53	1.53	0.2082
M5	20-04-1	15.90	606.00	4.25	3.72	5.97	2.19	1.58	1.51	2.51	1.41	1.41	0.2375


In [24]:
# 
columns = 'NP, MASTER, DATE, X, Y, CON, X*Y, INP, stdINP INPPX, IBGPX, EXP_TYPE, OPT'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE NP < 61
ORDER BY MASTER DESC
;
   """.format(columns))

head(columns)

i=0
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    i+=1
    
    
print('---\n{}'.format(i))

NP	MASTER	DATE	X	Y	CON	X*Y	INP	stdINP 	IBGPX	EXP_TYP	OPT
60	Q4	20-02-2	27.16	12.88	3.97	350.01	31.03	19.98	0.5038	-	0
60	Q4	20-02-2	28.07	13.13	3.82	368.74	30.71	22.48	0.4713	sh	0
40	Q4	20-02-2	16.01	10.42	3.31	166.75	9.30	12.81	0.4263	-	0
60	Q4	20-04-2	37.79	12.75	5.39	481.71	27.55	20.75	0.2537	-	1
40	Q4	20-04-2	33.70	12.84	5.93	432.52	33.27	27.78	0.2710	-	1
60	P3	20-02-2	4.96	5.39	8.28	26.76	4.66	2.03	0.2225	-	1
60	L3	20-04-1	35.59	11.53	2.98	410.30	20.13	10.84	0.3968	-	0
60	L3	20-04-1	43.08	11.60	3.18	499.49	21.01	14.20	0.3228	-	0
60	L3	20-04-1	53.05	10.22	3.08	541.97	15.79	9.24	0.2494	-	1
60	K5	20-03-1	16.75	12.42	5.82	208.07	12.01	7.96	0.3303	-	1
60	K5	20-03-1	18.03	13.11	5.26	236.34	11.28	7.93	0.2613	-	0
60	K5	20-03-1	15.82	13.54	5.60	214.28	10.76	6.41	0.2447	-	1
60	K5	20-03-1	18.35	13.11	5.89	240.54	15.70	10.47	0.3067	-	1
60	K5	20-03-1	16.16	13.58	5.37	219.52	9.60	6.25	0.2106	-	0
60	K4	20-03-1	20.51	11.91	3.11	244.28	7.49	5.49	0.2748	-	1
60	K4	20-03-1	21.92	11.36	2.77	248.98	3.9

In [54]:
# 
columns = 'NP, FILE, X, Y, CON, X*Y, INP, stdINP INPPX, IBGPX, EXP_TYPE'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE MASTER = 'P3' AND WAVELENGTH = 612 AND NP = 80

ORDER BY NP DESC
;
   """.format(columns))

head(columns)

i=0
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    i+=1
    
    
print('---\n{}'.format(i))

NP	FILE	X	Y	CON	X*Y	INP	stdINP 	IBGPX	EXP_TYP
80	raw_10_	5.06	5.98	7.86	30.26	4.53	2.27	0.1912	-
80	raw_29_	5.11	6.36	8.16	32.51	4.91	2.18	0.2032	sh
80	raw_32_	5.11	6.11	7.43	31.24	4.54	2.02	0.2021	strep
---
3


In [30]:
# 
columns = 'MEAS.ID, MASTER, NP, WAVELENGTH, FOLDER, FILE, RATE, X, Y, CON, X*Y, INP, stdINP INPPX, IBGPX, EXP_TYPE, OPT'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
WHERE MASTER = 'P3' AND NP=80 AND OPT=1
ORDER BY RATE DESC
;
   """.format(columns))

head(columns)

i=0
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    i+=1
    
    
print('---\n{}'.format(i))

MEAS.ID	MASTER	NP	WAVELEN	FOLDER	FILE	RATE	X	Y	CON	X*Y	INP	stdINP 	IBGPX	EXP_TYP	OPT
50	P3	80	612	20_02_2	raw_32_	168.45	5.11	6.11	7.43	31.24	4.54	2.02	0.2021	strep	1
36	P3	80	612	20_02_2	raw_10_	46.43	5.06	5.98	7.86	30.26	4.53	2.27	0.1912	-	1
44	P3	80	610	20_02_2	raw_11_	42.78	6.25	6.53	6.63	40.83	5.63	2.33	0.2363	-	1
49	P3	80	612	20_02_2	raw_29_	30.00	5.11	6.36	8.16	32.51	4.91	2.18	0.2032	sh	1
---
4


In [66]:
exp.show_all()

ID	DATE	MASTER_ID	DIP	FOLDER
1	20-02-25	2	615	20_02_25_P3/
2	20-02-26	6	727	20_02_26_Q3/
3	20-03-16	4	658	20_03_16_K5/
5	20-02-25	1	598	20_02_25_M5/
6	20-03-16	3	647	20_03_16_K4/
7	20-04-15	5	740	20_04_15_L3/
8	20-04-14	1	606	20_04_14_M5/
9	20-04-20	6	729	20_04_20_Q4/


In [96]:
con.commit()

In [12]:
folder = '20_04_20_Q4/'
file = 'raw_17_1'

columns = 'COUNT(MEAS.ID), MEAS.ID'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID 
WHERE FOLDER = '{}' AND FILE = '{}'
;
   """.format(columns, folder, file)
                  )

In [26]:
[d for d in data][0]

(3, 79)

In [20]:
b

In [28]:
columns = 'NP, WAVELENGTH, AR_TH, AR_NOISE, EXC_THS, AR_MIN, AR_COND, exp_type'
data = con.execute("""
SELECT {} FROM 'measurements'
WHERE WAVELENGTH=732
ORDER BY WAVELENGTH ASC;
   """.format(columns))

print(columns.replace(', ','\t'))
for row in data:
    print(str(row).replace(', ', '\t').replace("'", "")[1:-1])

NP	WAVELENGTH	AR_TH	AR_NOISE	EXC_THS	AR_MIN	AR_COND	exp_type
80	732	60.0	0.001	[2]	10	2	-
60	732	59.0	0.001	[2]	10	2	-
60	732	46.0	0.001	[2]	10	2	sh
40	732	12.0	0.001	[1.1]	2	2	-
80	732	100.0	0.001	[2]	2	2	-
60	732	20.0	0.001	[2]	2	2	-
80	732	40.0	0.001	[2]	2	2	strep
147	732	8.0	0.001	[2]	2	2	ps
40	732	9.0	0.001	[2]	6	2	-
80	732	31.0	0.001	[2]	6	2	sh


In [59]:
con.commit()

In [51]:
con.close()